<a href="https://colab.research.google.com/github/microprediction/precise/blob/main/inverse_covariance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [184]:
import numpy as np
n = 10
n_true = 50
X1 = np.random.randn(n,n_true)
E1 = np.cov(X1) # n x n
P1 = np.linalg.inv(E1)
n_train = 24
X2 = np.random.multivariate_normal(mean=np.zeros(n), cov = E1, size=n_train) # n_train x n
P1[:3,:3]

array([[ 1.28009093, -0.17315128,  0.04119847],
       [-0.17315128,  1.10831699,  0.14330659],
       [ 0.04119847,  0.14330659,  1.03479319]])

In [215]:
S = np.cov(np.transpose(X2)) # n x n 
P2 = np.linalg.inv(S)
err2 = np.linalg.norm(P1-P2)
err2

6.694635442300702

In [216]:
mu = np.squeeze(np.matmul(S,np.ones((n,1))))
mu

array([ 1.00026654,  1.21087137,  0.54289127, -0.10452896,  0.20019532,
        0.61348269,  2.90471695,  1.35524376,  0.53431303, -0.68546138])

In [217]:
D = np.diag(mu)
D[:3,:3]

array([[1.00026654, 0.        , 0.        ],
       [0.        , 1.21087137, 0.        ],
       [0.        , 0.        , 0.54289127]])

In [218]:
alpha2 = np.linalg.norm(E1-D)

In [219]:
beta2 = np.linalg.norm(E1-E2)

In [220]:
rho = alpha2/(alpha2+beta2)
Estar = rho*S + (1-rho)*D
Pstar = np.linalg.inv(Estar)
err_star = np.linalg.norm(P1-Pstar)
err_star, rho

(15.47216781097261, 0.6188792306002491)

In [221]:
Dinv = np.linalg.inv(D)
Dinv[:3,:3]

array([[0.99973353, 0.        , 0.        ],
       [0.        , 0.82585155, 0.        ],
       [0.        , 0.        , 1.84198947]])

In [225]:
A = rho/(1-rho)*np.matmul( Dinv, S )
I = np.eye(n)
np.linalg.norm( rho*S + (1-rho)*D - (1-rho)*np.matmul(D, (I + rho/(1-rho)*np.matmul(Dinv,S) )) )

3.920151220887736e-16

In [226]:
np.linalg.norm( Estar - (1-rho)*np.matmul(D, (I + rho/(1-rho)*np.matmul(Dinv,S) )) )

3.920151220887736e-16

In [227]:
Eprime = (1-rho)*np.matmul(D, (I + A ))
np.linalg.norm(Eprime-Estar)

3.920151220887736e-16

In [228]:
def hatE(S,rho:float, n_terms:int):
    """ 
      Estimate precision from empirical covar S and shrinkage rho 
    """
    n = np.shape(S)[0]
    mu = np.squeeze(np.matmul(S,np.ones((n,1))))
    D = np.diag(mu)
    Dinv = np.linalg.inv(D)
    return (1-rho)*np.matmul(D, (I + rho/(1-rho)*np.matmul(Dinv,S) ))
   
Ehat = hatE(S=S, rho=rho, n_terms=1)
np.linalg.norm(Estar-Ehat)

3.920151220887736e-16

In [276]:
def neumann(T, k):
   """  inv(I-T) via Neumann series """
   n = np.shape(T)[0]
   I = np.eye(n)
   Tk = I 
   Y = np.zeros(n)
   for j in range(0,k):
      Y = Y + Tk 
      Tk = np.matmul(Tk,T)
   return Y
    
C = np.array([[0,0.5,0.25],[5./7.,0,1./7.],[3./10.,3./5.,0.]])
Q1 = neumann(C,k=50)
Q2 = np.linalg.inv(np.eye(3)-C)
np.linalg.norm(Q1-Q2)


0.0003137031859386907

In [273]:
def hatP(S,rho,k):
    """ Well, this doesnt work """
    n = np.shape(S)[0]
    mu = np.squeeze(np.matmul(S,np.ones((n,1))))
    D = np.diag(mu)
    Dinv = np.linalg.inv(D)
    A = rho/(1-rho)*np.matmul(Dinv,S)
    print(np.linalg.norm(A))
    print(A[:3,:3])
    hatPtrue = 1/(1-rho)*np.matmul(np.linalg.inv(I + A ),Dinv  )
    hatPapprox = 1/(1-rho)*np.matmul(neumann(-A,k=k ),Dinv  )
    return hatPapprox

Phat = hatP(S=S,rho=rho, k=1)
np.linalg.norm(Phat-Pstar)

16.800920353026694
[[ 1.36085925 -0.09933846  0.29586379]
 [-0.08206069  1.22388106 -0.20744953]
 [ 0.54512325 -0.46269798  2.61838924]]


34.802077164859526

(7.9560681004217955, 0.8397639648277383)